# Importing libraries

In [1]:
import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import numpy as np

In [2]:
#For Dataset
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# for Dimensionality reduction
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA

# for evaluation
from sklearn.metrics import silhouette_score, mean_squared_error

In [3]:
# Colors for visualization

cols = ['#1FC17B', '#78FECF', '#555B6E', '#CC998D', '#429EA6',
        '#153B50', '#8367C7', '#EE6352', '#C287E8', '#F0A6CA', 
        '#521945', '#361F27', '#828489', '#9AD2CB', '#EBD494', 
        '#53599A', '#80DED9', '#EF2D56', '#446DF6', '#AF929D']

# Dataset creation

In [5]:
X, y = make_blobs(n_features = 50, centers = 20, n_samples = 20000,
                 cluster_std = 0.2, center_box = [-1,1], random_state = 17)

In [9]:
print('X: ', X.shape)
print('y: ', y.shape)

X:  (20000, 50)
y:  (20000,)


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 17)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)
print('y_train: ', y_train.shape)
print('y_test: ', y_test.shape)

X_train:  (18000, 50)
X_test:  (2000, 50)
y_train:  (18000,)
y_test:  (2000,)


# Baseline model: PCA